In [1]:
import requests
import pandas as pd
import numpy as np
import re
import os
from os import listdir
from os.path import isfile, join
import zipfile
import shutil

In [2]:
# manually join the misaligned surveys due to entry issues which is supposed to match 
# filter them out
# filter the subnational datasets out
# surveys in mismatch_filtered are those to be downloaded
df_download=(pd.read_csv('survey_to_download.csv')
#           create a column for unique identifier in the form of iso3c_survey_year
            .assign(filename = lambda x: x['ISO']+'_'+x['round']+'_'+x['year'])
            )
df_download

,ISO,round,region,country_x,year,status,reports,datasets,link,filename
0,AGO,MICS2,Eastern and Southern Africa,Angola,2001,Completed,Final,Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,AGO_MICS2_2001
1,ARG,MICS4,Latin America and Caribbean,Argentina,2011-2012,Completed,Final,Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,ARG_MICS4_2011-2012
2,BIH,MICS4,Europe and Central Asia,Bosnia and Herzegovina,2011-2012,Completed,Final,Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,BIH_MICS4_2011-2012
3,BIH,MICS3,Europe and Central Asia,Bosnia and Herzegovina,2006,Completed,Final,Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,BIH_MICS3_2006
4,BIH,MICS2,Europe and Central Asia,Bosnia and Herzegovina,2000,Completed,Final,Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,BIH_MICS2_2000
...,...,...,...,...,...,...,...,...,...,...
59,VNM,MICS3,East Asia and the Pacific,Viet Nam,2006,Completed,Final,Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,VNM_MICS3_2006
60,VNM,MICS2,East Asia and the Pacific,Viet Nam,2000,Completed,Final,Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,VNM_MICS2_2000
61,COD,MICS6,West and Central Africa,Democratic Republic of the Congo,2017-2018,Completed,"Snapshots,Survey findings",Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,COD_MICS6_2017-2018
62,COD,MICS4,West and Central Africa,Democratic Republic of the Congo,2010,Completed,"Final,Summary",Available,https://mics-surveys-prod.s3.amazonaws.com/MIC...,COD_MICS4_2010


In [3]:
# download the zip using links and extract them into proper folder

def download_df(df,no):
#   no is number of surveys we are downloading and unziping
    for i in range(0,no):
        url = df.loc[i,'link']
#       dir_new is folder for the survey
        dir_new = 'data_downloaded//'+ df.loc[i,'filename']
#       file_name is the zip downloaded
        file_name = 'data_downloaded//'+ df.loc[i,'filename']+'.zip'
#       micro_data is spss data folder
        micro_data = dir_new +'//microdata'
#       doc_data is documentation folder
        doc_data = dir_new +'//documentation'
    
#       Create folders if they don't already exist
        if os.path.exists('data_downloaded')==False:
             os.makedirs('data_downloaded')
        if os.path.exists(dir_new)==False:
             os.makedirs(dir_new)
        if os.path.exists(micro_data)==False:
             os.makedirs(micro_data)
        if os.path.exists(doc_data)==False:
             os.makedirs(doc_data)
                
#  Check if zip file for the survey already downloaded, if not download and save
        downloaded_zips = [f for f in listdir('data_downloaded') if isfile(join('data_downloaded', f))]
        if file_name not in downloaded_zips:
            r = requests.get(url, allow_redirects=True)
            open(file_name, 'wb').write(r.content)
            
#       Extract file into the survey folder  
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall(dir_new)
            
#       move documentation file into the documentation folder, other files in the microdata folder
        for root, dirs, files in os.walk(dir_new, topdown=False):
                for name in files:
                    if name.endswith('doc')|name.endswith('txt')|name.endswith('DOC'):
                        os.replace(os.path.join(root, name),os.path.join(doc_data, name))
                    else:
                        os.replace(os.path.join(root, name),os.path.join(micro_data, name))
                        
#           Delete the original extracted folder
                for dir in dirs:
                    if 'MICS' in dir:
                        shutil.rmtree(os.path.join(root, dir))

        os.replace(file_name,dir_new+'//'+ df.loc[i,'filename']+'.zip') 

In [4]:
# Yeah, we want to download all rows in df_download
download_df(df_download,df_download.shape[0])